# Time Series Simulation Study
In this notebook you make a simulation study to investigate the class of ARMA models. You will see that this class of models is 
- easily applicable with the `statsmodel` package,
- can be used for simulation of time series data,
- showing good forecasting abilities.

At the end of this notebook you will have learned:
- how to simulate time series data,
- how to define, estimate and forecast with ARMA and SARIMA models,
- how to decompose a time series into a trend, seasonality and rest,
- how to define Exponential Smoothing estimators and apply them to the data for forecasting,
- how to apply stationarty tests and
- how to apply ACF, PACF and Periodogram plots for model diagnosis.

## Load packages

In [ ]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from scipy import signal
%matplotlib inline

# If you want a style choose one
#plt.style.use('Solarize_Light2')
#plt.style.use('tableau-colorblind10')
NF_ORANGE = '#ff5a36'
NF_BLUE = '#163251'

See all matplotlib styles under [matplotlib styles](https://problemsolvingwithpython.com/06-Plotting-with-Matplotlib/06.13-Plot-Styles/)

## Define auxiliary functions

The following functions are defined to simplify time series plotting, time differencing and stationarity testing. We will use them through this notebook.

In [ ]:
def plot_ts(ts = None, ts_add = None, title ='Time Series', legend=['1']):
    """
    Plots one or two time series in a single plot
    
        Args:
        ts: 1d- or 2d-array of time series. Dimension
            must be (n,) or (n,2)
        title: Title for the time plot.
        legend: list of legend names. If empty no legend.
        
        Returns:
        matplotlib plot object
    """
    plt.figure(figsize=(10, 4))
    plt.plot(ts[:,], color=NF_ORANGE)
    plt.grid(True,axis='y')
    plt.title(title)
    if ts_add is not None:
        plt.plot(ts_add, color=NF_BLUE)
    if len(legend) > 0:
        plt.legend(legend)
    plt.show()

def plot_acf_pacf(ts, lags=10, layout='h'):
    """
    Plots the empirical ACF and PACF of a time series process.
    
        Args:
        ts: array of time series
        
        Returns:
        matplotlib subplot with ACF and PACF
    """
    if layout == 'h':
        fig, ax = plt.subplots(1, 2, figsize = (10,3))
    else: 
        fig, ax = plt.subplots(2, 1, figsize = (10,3))
    sm.tsa.graphics.plot_acf(ts,color = NF_ORANGE,lags=lags, ax = ax[0])
    sm.tsa.graphics.plot_pacf(ts,color = NF_ORANGE, lags = lags, ax = ax[1])    


def diff_series(ts, interval=1):
    """
    Differences a time series by a certain lag.
    
        Args:
        ts: array of 1d time series
        
        Returns:
        Differenced time series
    """
    diff = ts[interval:] - ts[:-interval]
    return diff

def kpss_test(ts):
    """
    Performs a KPSS test for the null hypothesis of stationarity.
    
        Args:
        ts: 1d time series
        
        Returns:
        Summary of test statistic and critical values
    """
    print ('Results of KPSS Test:')
    kpsstest = kpss(ts, regression='c', nlags='legacy')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    print (kpss_output)

def adf_test(ts):
    """
    Performs a Dickey-Fuller test for the null hypothesis of
    non-stationarity.
    
        Args:
        ts: 1-d time series
    
        Returns:
        Printed test statistic and critical values.
    """
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(ts, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used',
                                             'Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

Parameter estimates all resemble closely the true parameters in the DGP above and show high significance. In this case we have some complex roots (the only real one is greater than 1 and therefore we do not have a unit root). 

## Trend and seasonality
-------------------
As already mentioned in lecture, trends and seasonalities make a time series process non-stationary as they induce time-dependent distributions, i.e. values of $X_t$ occurring between $t$ and $t+h$ are differently distributed than values between times $t+k$ and $t+k+h$. In such cases the standard theory cannot be applied as stationary processes are assumed in this theory. 

It therefore needs a detrending and removing of seasonal components. In the following we consider (1) the composition of time series by a trend a seasonal component and a time-independent dynamic process. For this reason we sample data from a normal distribution (our white noise) and define two trend functions 

$$
\begin{align}
m_t&=\alpha_1+\alpha_2\cdot t\\
m_t&=\beta_1+\beta_2\cdot t + \beta_3\cdot t^2\\
\end{align}
$$

that we add to the white noise.

In [ ]:
# Define a DGP with a linear and quadratic trend
np.random.seed(42)

# Sample a normal white noise 
y = np.random.normal(0.0, 0.4, 100)

# Construct the time series with linear and quadratic trend
trend = 0.1 + 0.07 * np.arange(0,100)
y_ln = trend + y
y_qd = y_ln + 0.001 * np.arange(0,100)**2

# Plot the series
plot_ts(y_ln, y_qd, title = 'Trends', legend=['Linear', 'Qaudratic'])

### Estimate a quadratic trend
You learned that there are two methods to eliminate the trend in a time series. The first was estimating it and eliminate this estimate from the original time series. The other method is time differencing. In what follows we use the first method to eliminate the trend from the time series. 

We define a white noise process with 300 observations and compose a time series process by a qaudratic trend component with parameters $\alpha_1=0.2$, $\alpha_2=0.01$ and $\alpha_3=0.001$ and let the innovations be white noise normally distributed.

In [ ]:
# Define the data generating process
np.random.seed(42)
error = np.random.normal(0.0, 1.1, 300)
trend = 0.2 + 0.01 * np.arange(1, 301) + 0.001 * np.arange(1, 301)**2
y = trend + error

# Plot the time series
plot_ts(y,title='', legend=['Quadratic Trend'])

How can we estimate this trend? The most obvious approach is to use OLS with polynomial features in $t$:

$
x_t=\alpha_1+\alpha_2 t + \alpha_3 t^2
$

to estimate the coefficients $\alpha_i~,i=1,2,3$. 

In [ ]:
# Estimate a linear regression model 

# Start by constructing a time index variable and its square
X = np.array([np.arange(1, 301),np.arange(1, 301)**2]).T

# Use a constant term in the OLS regression
X = sm.tools.tools.add_constant(X)
model = sm.OLS(y, X)
res = model.fit()
print(res.summary())

Surprisingly, the constant term is highly non-significant and with a farout estimate. However, the coefficients for the time index and its square are near the true values of $[0.01, 0.001]$. To see, if the predictions are good enough we plot the time series together with the estimated function.

In [ ]:
# Plot the original and estimated time series
y_est = res.predict(X)
plot_ts(y, y_est, title='', legend=['Orig.', 'Est.'])

That appears to be a good estimate of the quadratic trend immanent in the time series. 

>__Exercise__: Take the original time series and difference the estimated one. Plot your results. What do you see? 

### Seasonality
Next to trends, seasonality has an influence on the distribution(s) of a time series, as seasonality implies that at certain times the process explores other value ranges than at others. This again would be a non-stationary setting and would make the standard theory for time series modeling to fail. 

By eliminating seasonalities (any cyclical pattern) from the time series data such a non-stationary time process can be transformed to a stationary one. Similar to trend elmination also seasonalities can be removed by two main approaches: (1) modeling the cyclical pattern and removing an estimate of this model from the original time series or (2) differencing the original time series at periodical time indices (e.g. a quarterly cyclical pattern has been finished after 4 time steps if data is quarterly),

$
\Delta(4)x_t=x_t-x_{t-4}~.
$

In the following we model the cyclical pattern by a _Fourier Series_ with a single sinus function and period 60 (e.g. 60 days).

In [ ]:
# Define the DGP
np.random.seed(42)

# Define the white noise process
error = np.random.normal(0.0, 1.1, 300)

# Define the seasonal pattern
# Use Fourier Series with a single sinus pattern
# with period 60.
seasonal = np.sin((2*np.pi/60)*np.arange(1,301))

# Combine the two terms and plot the series
y = seasonal + error
plot_ts(y, title = 'Seasonality',legend=['Cycle d=60'])

The cyclical pattern in the time series can be seen clearly. It is also easy to detect that the period of the cyclical pattern is 60 as after 60 time steps the process has arrived at a similar level as where it was before 60 steps. 

>__Exercise__: Model a cyclical pattern by an OLS equation using sinusoidal function of the time. Estimate this model and plot it together with the original time series process seen above. If the estimate is good enough in your eyes, difference this estimate from the original time series and plot the remaining term. 

#### Trend and seasonal component
What we usually observe or encounter in time series is a mixture of both, i.e. some trend and some cyclical pattern (sometimes not even easily detectable). In the following we generate a time series with such properties and you will decompose the model into its different components, i.e. (1) trend, (2) seasonal, and (3) residuals. Residuals might show no white noise properties (e.g. autocorrelation) and that would then need to be modeled as well (e.g. with an ARMA model). 

In the following you create a time series with a normally distributed innovation vector together with a sinuoidal seasonal pattern and a linear trend. 

In [ ]:
# Generate the data with trend and seasonal component
np.random.seed(42)

# Define the white noise process
error = np.random.normal(0.0, 1.1, 300)

# Define the seasonal term
seasonal = np.sin((2*np.pi/60)*np.arange(1,301))

# Define the trend component
trend = 0.01 + 0.05 * np.arange(1, 301) 

# Add all components together and plot them
y = trend + seasonal + error
plot_ts(y, title='Time series with trend and seasonality', legend=[])

This looks already like time series we might observe in the news or in business. As the decomposition is part of time series EDA, there exist pre-defined functions to decompose time series into their three components. In the `statsmodels` module the function `tsa.seasonal_decompose()` can be used to make such a decomposition. For the cyclical component you have to define a period length. (Take also a look at the multiplicative version of it). 

In [ ]:
# Decompose the time series into trend, seasonal component and residuals
y_dec = sm.tsa.seasonal_decompose(y, model = 'additive', period=60)
y_dec.plot()
plt.show();

The residuals show white noise behavior. And to see, if residuals have really no autocorrelations, we could do some diagnostic checks and tests. 

>__Exercise__: For the residuals process from the decomposition in `y_dec`. Plot the ACF and PACF. Take also a look at the [_Ljung-Box-Test_](https://www.statsmodels.org/stable/generated/statsmodels.stats.diagnostic.acorr_ljungbox.html) and apply this test to your residuals.

#### Example with a composed model

Remember: any stationary zero-mean time series can be expressed as a harmonic sum of (infinitely many) sinus and cosinus waves (very seldom this sum is finite - therefore modeling with such a harmonic mean a whole time series is avoided). The periodicity $d$ of a seasonal trend then determines the rate at which a cycle is finished in one time step:

$\frac{2\pi}{d}t$

We can also say $\frac{1}{d}$ is the frequency of the cycle, i.e. if $d=4$ a quarter of the whole cycle is concluded at a single time step. Trigonometrically, the cycle concludes $0.25\cdot2\pi$ _radiants_ per time step and a whole cycle after 4 time steps.

In the following simulation we model the seasonal component as a Fourier Series with a single sinus function and a periodicity of 4. A linear trend is added ogether with an AR(2) process. Note that in this case the white noise is generated inside of the `generate_sample()` method. The innvoation term $\varepsilon$ therewith is white noise, however, the residuum of a time series decomposition will be not, as this process is then an AR(2) process such that residuals from the time decomposiion would be autocorrelated (the Ljung-Box test can be applied here).

In [ ]:
# Generate an AR(2) process
np.random.seed(42)
ar_proc = sm.tsa.ArmaProcess(ar = [1, -.7, -.2]).generate_sample(nsample=1000)

# Define trend and seasonal component
seasonal = 1.3 * np.sin(2*np.pi/4*np.arange(1,1001))
trend = 0.01 + 0.02 * np.arange(1, 1001) 
y = trend + ar_proc + seasonal
plot_ts(y,legend=[],title='Composed time series')

You can see that the cycle is somehow still apparent, even though it got washed out a little by the AR(2) component. 

In a real-world time series approach the first step after looking at the graph above, would be a trend elimination. This time we use the second method for trend removal, i.e. the time differencing approach: 

$
\Delta x_t=x_t - x_{t-1}~.
$

Time differencing works well, if the trend is linear. With a non-linear trend we usually have to difference multiple times to arrive at a stationary process. 

In [ ]:
# Detrend the series
y_diff = diff_series(y)
plot_ts(y_diff, title = '', legend=['1st Diff.'])

In [ ]:
# We use for the sampling frequency 1 as we want to
# discover cycles over the original time steps.
f, Pxx = signal.periodogram(y_diff, fs = 1, window='hamming', scaling='spectrum')
plt.plot(f, Pxx, color = NF_ORANGE)
plt.title('Periodogram')
plt.show();

The dominant frequency here identified by the periodogram lays at around $\frac{1}{d}=0.25$ which corresponds to a periodicity of $d=4$, exactly the periodicity of the simulated seasonality. We can also observe that there exist many more cyclical patterns with other frequencies (observe the little trembling along the line), however the one at frequency 0.25 is the by far most dominant one. By using the periodogram we can identify what frequencies are included in our time series and which are the most dominant ones. By eliminating the most dominant ones we arrive at some point at a stationary time series that allows us explicit modeling via an ARMA model for example. 

As the periodicity in our time series appears to be at 4 (identified by the periodogram), we clean our series from this cyclical pattern by taking a difference between the actual value and its lag four time periods before:

$
\Delta(4) x_t=x_t-x_{t-4}~.
$

In [ ]:
# Difference the time series and plot
y_diff_4 = diff_series(y_diff, 4)
plot_ts(y_diff_4[:200,], title='', legend=['Diff(1,4)'])

From the graph it appears that the dominant cycle which was clearly visual has been removed from the series. If this is the case should be checked by a periodogram on the cleaned time series and also a stationarity test. We look at such stationarity tests in the next section. 

#### Testing for unit roots (non-stationarity)
After differencing to eliminate trend and seasonal components, we want to check, if we were able to arrive at a stationary time series. There exist some main procedures for testing on stationarity, namely [Philipps Perron](https://en.wikipedia.org/wiki/Phillips%E2%80%93Perron_test#cite_note-2) test, [augmented Dickey-Fuller (ADF)](https://en.wikipedia.org/wiki/Augmented_Dickey%E2%80%93Fuller_test) test, and the [Kwiatkowski-Phillips-Schmidt-Shin (KPSS)](https://en.wikipedia.org/wiki/KPSS_test). 

In contrast to the Phillips-Perron and augmented Dickey-Fuller the KPSS test the other way around, i.e. it tests on (trend-)stationarity, where trend-stationarity is the property of a time series to be stationary after elimination of a linear trend (by e.g. first-differencing it). The other two tests test the null of non-stationarity (unit-root).

Here we only apply the augmented Dickey-Fuller and KPSS tests to the data.

In [ ]:
# Performa an augmented Dickey-Fuller test on non-stationarity.
adf_test(y_diff_4)

As the $p$-value is clearly below 1% we can reject the null hypothesis of non-stationarity. 

To be sure we make also an KPSS test that tests the other way around: a null hypothesis of (trend-)stationarity. 

In [ ]:
# Perform KPSS test on stationarity
kpss_test(y_diff_4)

The p-value is 10% and we cannot reject the null hypothesis of stationarity for our differenced time series. 

In regard to the tests we have with high certainty a time series that is indeed stationary. With stationary time series we can now proceed our time series analysis and estimate an ARMA model. 

>__Exercise__: Apply also the Phillips-Perron test and write a corresponding test function as in `kpss_test()` or `adf_test()` to report results. 

## Exponential Smoothing
As mentioned in lecture a very strong benchmark class of time series models is the class of exponential smoothing models, where the actual values are modeled as exponential averages between the last value and past values:

$
\hat{x}_{t+1}=\alpha x_t + (1-\alpha)\hat{x}_{t-1}~.
$

These exponential smoothers are simple, but hard to beat. 

### Import the exponential smoothing functions
We will use a very simple exponential smoothing method and then the well-known [Holt-Winters](https://otexts.com/fpp2/holt-winters.html) smoother.

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.api import SimpleExpSmoothing

### Simple exponential smoothing without seasonality and trend
We use in the following the time series process simulated above for the SARIMA model. The simple exponential smoothing method takes only an average over past values and does assume no trend or seasonality in the data. We should expect that it does not perform well. 

In [ ]:
# Use a smoothing level alpha of 0.4, i.e.
# past values will be given a weight of 60%
# actual values a weight of the remaining 40% 
simple_exp = SimpleExpSmoothing(y, initialization_method='estimated').fit(smoothing_level=0.6,optimized=False)

# Predict the last 20 values in the original
# time series
pred = simple_exp.predict(980, 999)
y_pred = np.ndarray((100,))

# Set the points not forecasted to NaN 
# (this does not plot them)
y_pred[:80] = np.NaN 
y_pred[80:] = pred

# Print the forecast RMSE.
print('In-sample RMSE: {}'.format(np.sqrt(np.sum((y[980:] - pred)**2))))
plot_ts(y[900:], y_pred, title = '', legend=['Orig.', 'Est.'])

As expected the simple exponential smoother that does not take into account the linear trend apparent in the simulated time series data does bad. We have an RMSE of around 7.5 (whereas the RMSE for the SARIMA model was at 4.2. We see that the exponential smoother exhibits some forerunning, which is caused by neglecting two string dynamics in the simulated time series. 

### Holt-Winters exponential smoothing with trend and seasonality
Holt-Winters is a smoothing approach that is usually very hard to beat in practice. Let us see what the smoother can achieve on this simulated data. In contrast to the simple exponential smoother, the Holt-Winters method accounts for a trend and a seasonal component in the data. We can choose either an _additive_ or _multiplicative_ approach. As we do know that the trend and seasonal component have been added and not multiplicated to the white noise we choose the additive method `add` for these two components of the Holt-Winters smoother. 

In [ ]:
# Use an additive seasonality and additive trend. 
# As we know from prior EDA, we have a cycle of 4. 
holt_winters = ExponentialSmoothing(y, seasonal_periods=4,seasonal='add', trend='add', initialization_method='estimated')
res = holt_winters.fit()

# Make a prediction for the last 20 values. 
pred = res.predict(980, 999)
y_pred = np.ndarray((100,))

# Set the points not forecasted to NaN 
# (this does not plot them)
y_pred[:80] = np.NaN 
y_pred[80:] = pred

# Print the forecast RMSE.
print('In-sample RMSE: {}'.format(np.sqrt(np.sum((y[980:1000] - pred)**2))))
plot_ts(y[900:], y_pred, title = '', legend=['Orig.', 'Est.'])

As expected the Holt-Winters exponential smoothing performs well.